# Stochastic Network Optimisation with PyPSA

https://pypsa--1250.org.readthedocs.build/en/1250/examples/stochastic-optimization/#two-stage-stochastic-programming

In [7]:
import matplotlib.pyplot as plt
import pandas as pd
from linopy.expressions import merge
from xarray import DataArray

import pypsa
from pypsa.components.common import as_components

In [8]:
print(pypsa.__version__)

0.35.0.post1.dev106+g2c22e7f


## Toy problem

In [9]:
# Scenario definitions - Gas price uncertainty
SCENARIOS = ["low", "med", "high"]
GAS_PRICES = {"low": 40, "med": 70, "high": 100}  # EUR/MWh_th
PROB = {"low": 0.4, "med": 0.3, "high": 0.3}  # Scenario probabilities
BASE = "low"  # Base scenario for network construction

# System parameters
FREQ = "3h"  # Time resolution
LOAD_MW = 1  # Constant load (MW)
SOLVER = "highs"  # Optimization solver

# Time series data URL
TS_URL = (
    "https://tubcloud.tu-berlin.de/s/pKttFadrbTKSJKF/download/time-series-lecture-2.csv"
)

# Load and process time series data
ts = pd.read_csv(TS_URL, index_col=0, parse_dates=True)
ts = ts.resample(FREQ).asfreq()  # Resample to 3-hour resolution


# Technology specifications
def annuity(life, rate):
    """Compute capital recovery factor for annualizing investment costs."""
    return rate / (1 - (1 + rate) ** -life) if rate else 1 / life


# Technology data: investment costs, efficiencies, marginal costs
TECH = {
    "solar": {"profile": "solar", "inv": 1e6, "m_cost": 0.01},
    "wind": {"profile": "onwind", "inv": 2e6, "m_cost": 0.02},
    "gas": {"inv": 7e5, "eff": 0.6},
    "lignite": {"inv": 1.3e6, "eff": 0.4, "m_cost": 130},
}

# Financial parameters
FOM, DR, LIFE = 3.0, 0.03, 25  # Fixed O&M (%), discount rate, lifetime (years)

# Calculate annualized capital costs
for cfg in TECH.values():
    cfg["fixed_cost"] = (annuity(LIFE, DR) + FOM / 100) * cfg["inv"]

## Helper functions for visualisation

In [10]:
# Color mapping for visualizations
COLOR_MAP = {
    "solar": "gold",
    "wind": "skyblue",
    "gas": "brown",
    "lignite": "black",
}


def plot_capacity(
    df,
    title="Capacity Mix",
    xlabel="Scenario",
    ylabel="Capacity (MW)",
    figsize=(8, 4),
    color_map=None,
    rotation=0,
):
    """Plot capacity mix as stacked bar chart"""
    if color_map is None:
        color_map = COLOR_MAP

    colors = [color_map.get(c, "gray") for c in df.columns]
    ax = df.plot(kind="bar", stacked=True, figsize=figsize, color=colors)
    ax.set(title=title, xlabel=xlabel, ylabel=ylabel)
    ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.xticks(rotation=rotation, ha="right" if rotation > 0 else "center")
    plt.tight_layout()
    return ax


def plot_cost(
    series,
    title="Total Cost",
    xlabel="Scenario",
    ylabel="Cost (EUR/year)",
    figsize=(6, 4),
    rotation=0,
):
    """Plot costs as bar chart"""
    ax = series.plot(kind="bar", figsize=figsize, color="steelblue")
    ax.set(title=title, xlabel=xlabel, ylabel=ylabel)
    # Format y-axis as millions
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"{x / 1e6:.1f}M"))
    plt.xticks(rotation=rotation, ha="right" if rotation > 0 else "center")
    plt.tight_layout()
    return ax

## Create PyPSA network for the toy problem

In [11]:
def build_network(gas_price):
    """
    Create PyPSA network with:
    - Single bus (DE) with constant load
    - Extendable generators: solar, wind, gas, lignite
    """
    n = pypsa.Network()
    n.set_snapshots(ts.index)
    n.snapshot_weightings = pd.Series(int(FREQ[:-1]), index=ts.index)  # 3-hour weights

    # Add bus and load
    n.add("Bus", "DE")
    n.add("Load", "DE_load", bus="DE", p_set=LOAD_MW)

    # Add renewable generators (variable renewable energy)
    for tech in ["solar", "wind"]:
        cfg = TECH[tech]
        n.add(
            "Generator",
            tech,
            bus="DE",
            p_nom_extendable=True,
            p_max_pu=ts[cfg["profile"]],  # Renewable availability profile
            capital_cost=cfg["fixed_cost"],
            marginal_cost=cfg["m_cost"],
        )

    # Add conventional generators (dispatchable)
    for tech in ["gas", "lignite"]:
        cfg = TECH[tech]
        # Gas marginal cost depends on gas price and efficiency
        mc = (gas_price / cfg.get("eff")) if tech == "gas" else cfg["m_cost"]
        n.add(
            "Generator",
            tech,
            bus="DE",
            p_nom_extendable=True,
            efficiency=cfg.get("eff"),
            capital_cost=cfg["fixed_cost"],
            marginal_cost=mc,
        )
    return n